<a href="https://colab.research.google.com/github/jain-abhinav02/Sentiment-Classfication/blob/master/Airline_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense,LSTM,GRU,Dropout,BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import os
from keras.initializers import Constant

Using TensorFlow backend.


In [0]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2019-06-03 11:30:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-06-03 11:30:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-06-03 11:30:28--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!ls
!pwd

drive		   glove.6B.200d.txt  glove.6B.50d.txt	sample_data
glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
/content


In [0]:
# processing the glove embedding txt file
GLOVE_DIR = ''
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
# import dataset into a pandas dataframe
tweets1 = pd.read_csv("drive/My Drive/Datasets/Tweets.csv")

In [0]:
# choosing a subset of the dataset to test models quickly
tweets = tweets1.iloc[:,:]
print(type(tweets))

<class 'pandas.core.frame.DataFrame'>


In [0]:
# a glimpse of the dataset
print(tweets.shape)
tweets.head()

(14640, 15)


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [1]:
# Airline wise tweets count
air_wise_tweets = tweets['airline'].value_counts()
print(air_wise_tweets)
print()
print(air_wise_tweets.plot.bar())

NameError: ignored

In [0]:
# Overall count in each of the sentiment classes 
# positive, negative, neutral
print(tweets.airline_sentiment.value_counts())

In [0]:
# Sentiment airline wise
air_wise_sent = tweets.groupby('airline').airline_sentiment.value_counts().unstack().fillna(0)
print(air_wise_sent)
print(air_wise_sent.plot.bar(stacked=False))

In [0]:
# One hot encode the output labels ( positive, negative , neutral)
y = tweets.iloc[:,1:2].values
encoder = OneHotEncoder()
y = encoder.fit_transform(y).toarray()

In [0]:
# Extract the tweets column and remove the prefix @airline_name
x = tweets['text']
x= x.str.extract('\w\s([\s\S]*)',expand=True)
x1 = x.iloc[:,0].values

In [0]:
# Find the maximum length of any review
print('Maximum review length: {}'.format(len(max(x1, key=len))))

Maximum review length: 175


In [0]:
maxlen=150
embedding_size = 100
MAX_NUM_WORDS = 20000

In [0]:
# Tokenize the text reviews to integer arrays
tk = Tokenizer(lower = True)
tk.fit_on_texts(x1)
x_seq = tk.texts_to_sequences(x1)
x_pad = pad_sequences(x_seq,maxlen=maxlen)

In [0]:
vocab_size = len(tk.word_counts.keys())+1

In [0]:
# Counting the number of unique words in all reviews
word_index = tk.word_index
print('Found %s unique tokens.' % len(word_index))

Found 15670 unique tokens.


In [0]:
# preparing embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
# splitting training and test data
x_train,x_test,y_train,y_test = train_test_split(x_pad,y,test_size=0.25,random_state=1)

In [0]:
# splitting training and validation data
valid_size = 64
x_train1 = x_train[valid_size:]
y_train1 = y_train[valid_size:]
x_val = x_train[:valid_size]
y_val = y_train[:valid_size]

In [0]:
print(x_train.shape)

(10980, 150)


In [0]:
# create a RNN model
model = Sequential()
#model.add(Embedding(vocab_size,embedding_size,embeddings_initializer=Constant(embedding_matrix),input_length = maxlen,trainable=False))
#model.add(LSTM(100,return_sequences = True))
model.add(Embedding(vocab_size,embedding_size,input_length=maxlen))
model.add(LSTM(256))

model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(512,activation='relu'))

model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(3,activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# review the model
model.compile(loss= 'categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 100)          1567100   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
__________

In [0]:
# begin training on dataset
batch_size = 128
num_epochs = 5
model.fit(x_train1,y_train1,validation_data= (x_val,y_val),batch_size = batch_size, epochs= num_epochs)

Train on 10916 samples, validate on 64 samples
Epoch 1/5
10916/10916 [==============================] - 20s 2ms/step - loss: 0.1521 - acc: 0.9470 - val_loss: 1.9061 - val_acc: 0.5938
Epoch 2/5
10916/10916 [==============================] - 20s 2ms/step - loss: 0.1262 - acc: 0.9555 - val_loss: 1.0015 - val_acc: 0.7812
Epoch 3/5
10916/10916 [==============================] - 20s 2ms/step - loss: 0.1067 - acc: 0.9644 - val_loss: 1.0825 - val_acc: 0.7500
Epoch 4/5
10916/10916 [==============================] - 20s 2ms/step - loss: 0.0961 - acc: 0.9697 - val_loss: 1.3066 - val_acc: 0.7344
Epoch 5/5
10916/10916 [==============================] - 20s 2ms/step - loss: 0.0896 - acc: 0.9712 - val_loss: 1.0048 - val_acc: 0.7969


In [0]:
# check accuracy on test data
scores = model.evaluate(x_test,y_test,verbose=0)
print("accuracy:",str(scores[1]))

accuracy: 0.7401639344262295


In [0]:
u=8
print(tk.sequences_to_texts(x_test[u:u+1]))
print(model.predict(x_test[u:u+1]))

['what a mess caused by the computer systems flight 719 in 3 hours late flight and now no gate for us est 26 min wait']
[[9.9999988e-01 1.2177122e-07 4.4750756e-08]]
